In [2]:
from ultralytics import YOLO
from ultralytics.yolo.v8.detect.predict import DetectionPredictor
import cv2
import pytesseract
import numpy as np
import time
import pyttsx3
import os
from gtts import gTTS
from playsound import playsound
import onnx
import torch

In [3]:
model=YOLO("yolov8l.pt")

100%|██████████| 83.7M/83.7M [00:23<00:00, 3.81MB/s]


In [3]:
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [4]:
frames_since_last_detection = {}
previous_detected_objects=[]

In [6]:
output_layers = model.getUnconnectedOutLayersNames()
detected_objects = set()

AttributeError: 'YOLO' object has no attribute 'getUnconnectedOutLayersNames'. See valid attributes below.

    YOLO (You Only Look Once) object detection model.

    Args:
        model (str, Path): Path to the model file to load or create.
        task (Any, optional): Task type for the YOLO model. Defaults to None.

    Attributes:
        predictor (Any): The predictor object.
        model (Any): The model object.
        trainer (Any): The trainer object.
        task (str): The type of model task.
        ckpt (Any): The checkpoint object if the model loaded from *.pt file.
        cfg (str): The model configuration if loaded from *.yaml file.
        ckpt_path (str): The checkpoint file path.
        overrides (dict): Overrides for the trainer object.
        metrics (Any): The data for metrics.

    Methods:
        __call__(source=None, stream=False, **kwargs):
            Alias for the predict method.
        _new(cfg:str, verbose:bool=True) -> None:
            Initializes 

In [6]:
engine = pyttsx3.init()

In [7]:
cap = cv2.VideoCapture(0)

In [8]:
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

True

In [9]:
def post_process(frame, outputs, conf_threshold, nms_threshold):
    height, width, _ = frame.shape
    boxes = []
    confidences = []
    class_ids = []
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w // 2
                y = center_y - h // 2
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    return [boxes[i] for i in indices], [confidences[i] for i in indices], [class_ids[i] for i in indices]


In [ ]:
while True:
    ret, frame = cap.read()
    conf_threshold = 0.5  # Confidence threshold for filtering out weak detections
    nms_threshold = 0.4   # Non-maximum suppression threshold for eliminating overlapping boxes

    # Detect objects in the frame
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    boxes, confidences, class_ids = post_process(frame, outputs, conf_threshold, nms_threshold)

    # Update frames_since_last_detection
    for i in range(len(frames_since_last_detection)):
        object_name = list(frames_since_last_detection.keys())[i]
        if object_name in detected_objects:
            frames_since_last_detection[object_name] = 0
        else:
            frames_since_last_detection[object_name] += 1

    # Check for new detections and speak the object name
   # Check for new detections and speak the object name
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    for i in range(len(indices)):
        index = indices[i]
        class_id = class_ids[index]
        object_name = classes[class_id]
        if object_name not in detected_objects:
            detected_objects.add(object_name)
            engine.say(f"{object_name} detected")
            engine.runAndWait()
        frames_since_last_detection[object_name] = 0

    # Draw bounding boxes around the detected objects
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    if len(indices) == 0:
        continue
    for i in indices:
        i = indices[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)


    cv2.imshow("Object Detection", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image to make it binary
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)

        # Make sure the contour is big enough to be a hand
        if cv2.contourArea(contour) > 10000:
            # Find the convex hull of the hand
            hull = cv2.convexHull(contour)

            # Find the defects in the convex hull
            defects = cv2.convexityDefects(contour, cv2.convexHull(contour, returnPoints=False))

            # Count the number of fingers
            num_fingers = 0
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])

                # Calculate the length of the sides of the triangle formed by the defects
                a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

                # Calculate the angle between the sides using the law of cosines
                angle = np.arccos((b**2 + c**2 - a**2) / (2*b*c)) * 180 / np.pi

                # If the angle is less than 90 degrees, it's a finger
                if angle < 90:
                    num_fingers += 1

            # Draw the contour and hull
            cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
            cv2.drawContours(frame, [hull], -1, (0, 0, 255), 2)

            # Display the number of fingers
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, str(num_fingers), (50, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Hand Gesture Recognition', frame)

    # Exit on ESC
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'


In [2]:
from ultralytics import YOLO
import pandas as pd
import cv2
import pyttsx3
import pytesseract
import os
from gtts import gTTS
from playsound import playsound

model = YOLO("yolov8l.pt")


with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

engine = pyttsx3.init()

cap = cv2.VideoCapture(0)
detected_objects = set()
while True:
    ret, frame = cap.read()

    # Perform object detection
    results = model(frame)
    print(results)
    # Get the detected objects and their labels
#     objects = pd.DataFrame(results).xyxy[0][0]
#     labels = objects["name"].tolist()

#     # Check for new detections and speak the object name
#     for label in labels:
#         if label not in detected_objects:
#             detected_objects.add(label)
#             engine.say(f"{label} detected")
#             engine.runAndWait()

    # Draw bounding boxes around the detected objects
    frame = results.render()

    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 2841.2ms
Speed: 12.1ms preprocess, 2841.2ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: '

AttributeError: 'list' object has no attribute 'render'

In [ ]:
from ultralytics import YOLO
import cv2
import pyttsx3
import pytesseract
import os
from gtts import gTTS
from playsound import playsound

model = YOLO("yolov8l.pt")

engine = pyttsx3.init()
results= model.predict(source="0",show=True)
results=list(results)



1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

0: 480x640 1 person, 3031.7ms
0: 480x640 1 person, 2848.2ms
0: 480x640 1 person, 1967.9ms
0: 480x640 1 person, 2129.1ms
0: 480x640 1 person, 1 tie, 2344.0ms
0: 480x640 1 person, 2103.4ms
0: 480x640 1 person, 1887.8ms
0: 480x640 1 person, 2358.2ms


In [4]:
from ultralytics import YOLO
import cv2
import pyttsx3

model = YOLO("yolov8l.pt")

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Perform object detection on live webcam feed
model.predict(source="0",show=True,)

# Convert the results to a list
# results = list(results)

# # Process the results
# for result in results:
#     # Extract the labels and bounding boxes
#     labels = result["label"]
#     boxes = result["box"]

#     # Print the labels and bounding boxes
#     for label, box in zip(labels, boxes):
#         print("Label:", label)
#         print("Bounding Box:", box)
#         print()

# Shut down the text-to-speech engine
engine.stop()



1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

0: 480x640 1 person, 2763.5ms
0: 480x640 1 person, 2680.1ms
0: 480x640 1 person, 2469.8ms
0: 480x640 1 person, 1744.0ms
0: 480x640 1 person, 1795.1ms
0: 480x640 1 person, 2038.5ms
0: 480x640 1 person, 2505.9ms
0: 480x640 1 person, 2552.7ms
0: 480x640 1 person, 2643.3ms
0: 480x640 1 person, 2539.9ms
0: 480x640 1 person, 2527.8ms
0: 480x640 1 person, 1 cell phone, 2507.7ms
0: 480x640 1 person, 1 c

KeyboardInterrupt: 

In [6]:
from ultralytics import YOLO
import cv2
import pyttsx3

model = YOLO("yolov8l.pt")

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Set the desired FPS (frames per second)
desired_fps = 15

# Calculate the frame delay based on desired FPS
frame_delay = int(1000 / desired_fps)

# Create a set to keep track of detected objects
detected_objects = set()

# Function to check if an object has been detected and announce it
def announce_object(object_label):
    if object_label not in detected_objects:
        detected_objects.add(object_label)
        engine.say(f"{object_label} detected.")
        engine.runAndWait()

# Perform object detection on live webcam feed
cap = cv2.VideoCapture(0)

# List to store detected objects
detected_object_list = []

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    if not ret:
        break

    # Perform object detection on the frame
    results = model(frame)

    # Process the results
    for result in results.pred:
        for detection in result:
            label = detection["name"]
            confidence = detection["confidence"]
            bbox = detection[0:4].astype(int)

            # Only consider detections with high confidence
            if confidence > 0.5:
                announce_object(label)
                detected_object_list.append(label)

            # Draw bounding box on the frame
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (bbox[0], bbox[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Object Detection", frame)

    # Check for 'q' key press to exit
    if cv2.waitKey(frame_delay) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

# Shut down the text-to-speech engine
engine.stop()

print("Detected Objects:")
for obj in detected_object_list:
    print(obj)



0: 480x640 2 persons, 2909.7ms
Speed: 8.0ms preprocess, 2909.7ms inference, 11.0ms postprocess per image at shape (1, 3, 640, 640)


AttributeError: 'list' object has no attribute 'pred'

In [14]:
from ultralytics import YOLO
import cv2
import pyttsx3

model = YOLO("yolov8l.pt")

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Set the desired FPS (frames per second)
desired_fps = 15

# Calculate the frame delay based on desired FPS
frame_delay = int(1000 / desired_fps)

# Create a set to keep track of detected objects
detected_objects = set()

# Function to check if an object has been detected and announce it
def announce_object(object_label):
    if object_label not in detected_objects:
        detected_objects.add(object_label)
        engine.say(f"{object_label} detected.")
        engine.runAndWait()

# Perform object detection on live webcam feed
cap = cv2.VideoCapture(0)

# List to store detected objects
detected_object_list = []

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    if not ret:
        break

    # Perform object detection on the frame
    results = model(frame)

    # Print the attributes of the Results object
    print(results)

    # Process the results
    # Adjust the attribute access based on the available attributes
    # and their structure in the Results object
    for result in results.boxes:  # Example attribute access, adjust accordingly
        label = result["name"]
        confidence = result["confidence"]
        bbox = result[0:4].astype(int)

        # Only consider detections with high confidence
        if confidence > 0.5:
            announce_object(label)
            detected_object_list.append(label)

        # Draw bounding box on the frame
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 2)
        cv2.putText(frame, f"{label} {confidence:.2f}", (bbox[0], bbox[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Object Detection", frame)

    # Check for 'q' key press to exit
    if cv2.waitKey(frame_delay) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

# Shut down the text-to-speech engine
engine.stop()

print("Detected Objects:")
for obj in detected_object_list:
    print(obj)



0: 480x640 1 person, 2977.1ms
Speed: 10.9ms preprocess, 2977.1ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: '

AttributeError: 'list' object has no attribute 'boxes'

In [3]:
import torch
from models.experimental import attempt_load

# Load the YOLOv5 model
model = attempt_load("yolov8l.pt", map_location=torch.device("cpu"))

# Set the input size for the model
input_size = 640  # Adjust this according to the model's input size

# Create a sample input tensor with the desired input size
dummy_input = torch.zeros((1, 3, input_size, input_size), device=torch.device("cpu"))

# Export the model to ONNX format
onnx_path = "yolov8l.onnx"  # Specify the path to save the ONNX model
torch.onnx.export(model, dummy_input, onnx_path, opset_version=11)

print("YOLOv5 model successfully converted to ONNX format.")


ModuleNotFoundError: No module named 'models'